## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/tgeis/OneDrive/Desktop/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,52.48,99,29,2.37,scattered clouds
1,1,Longyearbyen,SJ,78.2186,15.6401,15.64,57,0,8.05,clear sky
2,2,Bambous Virieux,MU,-20.3428,57.7575,77.25,88,40,3.44,scattered clouds
3,3,Hermanus,ZA,-34.4187,19.2345,68.81,74,0,11.21,clear sky
4,4,Khatanga,RU,71.9667,102.5000,16.70,95,99,9.60,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bambous Virieux,MU,-20.3428,57.7575,77.25,88,40,3.44,scattered clouds
10,10,Maragogi,BR,-9.0122,-35.2225,81.91,75,56,11.72,broken clouds
13,13,Port-Gentil,GA,-0.7193,8.7815,85.96,74,20,11.50,few clouds
21,21,Rikitea,PF,-23.1203,-134.9692,76.26,73,35,7.40,scattered clouds
22,22,Hilo,US,19.7297,-155.0900,74.28,94,75,5.99,broken clouds
25,25,Port Blair,IN,11.6667,92.7500,80.64,89,20,2.30,mist
26,26,Mahibadhoo,MV,3.7833,72.9667,86.22,75,51,18.63,broken clouds
28,28,Puerto Ayora,EC,-0.7393,-90.3518,76.96,85,83,7.09,broken clouds
29,29,Acapulco,MX,16.8634,-99.8901,80.42,57,20,4.61,few clouds
32,32,Arraial Do Cabo,BR,-22.9661,-42.0278,84.16,61,0,11.50,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                263
City                   263
Country                260
Lat                    263
Lng                    263
Max Temp               263
Humidity               263
Cloudiness             263
Wind Speed             263
Current Description    263
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
clean_df.count()

City_ID                260
City                   260
Country                260
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bambous Virieux,MU,77.25,scattered clouds,-20.3428,57.7575,
10,Maragogi,BR,81.91,broken clouds,-9.0122,-35.2225,
13,Port-Gentil,GA,85.96,few clouds,-0.7193,8.7815,
21,Rikitea,PF,76.26,scattered clouds,-23.1203,-134.9692,
22,Hilo,US,74.28,broken clouds,19.7297,-155.0900,
25,Port Blair,IN,80.64,mist,11.6667,92.7500,
26,Mahibadhoo,MV,86.22,broken clouds,3.7833,72.9667,
28,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
29,Acapulco,MX,80.42,few clouds,16.8634,-99.8901,
32,Arraial Do Cabo,BR,84.16,clear sky,-22.9661,-42.0278,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... Skipping")

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   260
Country                260
Max Temp               260
Current Description    260
Lat                    260
Lng                    260
Hotel Name             260
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/tgeis/OneDrive/Desktop/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))